In [3]:
# ✅ Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# ✅ Load Required Libraries
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import pandas as pd
from sentence_transformers import SentenceTransformer, util
import gradio as gr

# ✅ Define Paths (Ensure the user has these in their Drive)
model_path = "/content/drive/My Drive/trained_resume_model"
datasets_path = "/content/drive/My Drive/datasets/job_description.csv"

# ✅ Load Trained Model & Tokenizer
device = "cuda" if torch.cuda.is_available() else "cpu"
model = AutoModelForSeq2SeqLM.from_pretrained(model_path).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_path)

# ✅ Load Job Descriptions
df_jobs = pd.read_csv(datasets_path)

# ✅ Load Sentence Transformer for Job Matching
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
job_texts = df_jobs["Job Description"].tolist()
job_embeddings = embedding_model.encode(job_texts, convert_to_tensor=True)
print(f"✅ Loaded {len(job_embeddings)} job embeddings")

# ✅ Define Job Matching Function

def find_best_match(resume_text):
    resume_embedding = embedding_model.encode(resume_text, convert_to_tensor=True)
    similarity_scores = util.pytorch_cos_sim(resume_embedding, job_embeddings)
    best_match_idx = torch.argmax(similarity_scores).item()
    return df_jobs.iloc[best_match_idx]["Job Title"], df_jobs.iloc[best_match_idx]["Job Description"]

# ✅ Define Resume Processing Function

def summarize_resume(resume_text):
    """Extracts key skills and experiences from a resume using the trained model."""
    input_text = f"Resume: {resume_text}\n"
    input_ids = tokenizer(input_text, return_tensors="pt", max_length=1024, truncation=True).input_ids.to(device)
    output_ids = model.generate(
        input_ids,
        max_length=350,
        temperature=0.82,
        top_k=40,
        top_p=0.9,
        do_sample=True
    )
    summary = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return summary

# ✅ Define Gradio Interface
def resume_evaluator(resume_text):
    matched_job, matched_description = find_best_match(resume_text)
    feedback = summarize_resume(resume_text)
    return matched_job, feedback

iface = gr.Interface(
    fn=resume_evaluator,
    inputs=gr.Textbox(label="Paste Your Resume"),
    outputs=[
        gr.Textbox(label="Best Matched Tech Job"),
        gr.Textbox(label="Resume Summary")
    ],
    title="AI Resume Evaluator",
    description="Upload your resume and get AI-powered feedback!"
)

# ✅ Launch Gradio App
iface.launch(share=True)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.11/dist-packages/transformers/models/bart/configuration_bart.py:176: UserWarning: Please make sure the config includes `forced_bos_token_id=0` in future versions. The config can simply be saved and uploaded again to be fixed.
  warnings.warn(


✅ Loaded 2277 job embeddings
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://2c98b29f464799c0a8.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
